In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Interim Data File Locations
interim_data = '../data/interim/'

In [3]:
df = pd.read_csv(interim_data + 'pos_neg_class_file.csv')
df.shape

(87550, 3)

In [4]:
df.sample(10)

,name_a,name_b,target
8878,Lila,Della,1
11323,Miki,Miroje,1
17164,Xena,Christine,1
30507,Chabel,Mile,0
16104,Tolya,Tolechka,1
63944,Caris,Bitse,0
16527,Varya,Varka,1
56055,Jarina,Si,0
48926,Ovidiu,Kimme,0
13126,Pave,Pavo,1


In [5]:
# Save output to interim data folder as csv file
df.to_csv(interim_data + 'eda_results.csv', index=False)